In [1]:
import requests
from bs4 import BeautifulSoup


def get_html(url):
    """
    Get and parse HTML content from a Wikipedia page

    :param url: str, the URL of the Wikipedia page to parse
    :return: bs4.BeautifulSoup object, the parsed HTML content of the Wikipedia page
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup


def extract_title(soup):
    """
    Extract the title of the Wikipedia article

    :param soup: bs4.BeautifulSoup object, the parsed HTML content of the Wikipedia page
    :return: str, the title of the Wikipedia article
    """
    title = soup.find('h1', id='firstHeading').text
    return title


def extract_text(soup):
    """
    Extract the text of the Wikipedia article, along with the headings for each section

    :param soup: bs4.BeautifulSoup object, the parsed HTML content of the Wikipedia page
    :return: dict, a dictionary containing the headings as keys and the text for each section as values
    """
    text = {}
    content = soup.find('div', id='content')
    body = content.find('div', id='bodyContent')
    for section in body.find_all(['h2', 'h3']):
        section_title = section.text
        section_text = ''
        for element in section.next_siblings:
            if element.name and element.name.startswith('h'):
                break
            if element.name == 'p':
                section_text += element.text
        text[section_title] = section_text
    return text


def extract_links(soup):
    """
    Collect all links in the Wikipedia page that redirects to another Wikipedia page

    :param soup: bs4.BeautifulSoup object, the parsed HTML content of the Wikipedia page
    :return: list, a list of all links that redirect to another Wikipedia page
    """
    links = []
    for link in soup.find_all('a'):
        if link.has_attr('href') and link['href'].startswith('/wiki/'):
            links.append('https://en.wikipedia.org' + link['href'])
    return links


def scrape_wikipedia(url):
    """
    Scrape text from a Wikipedia page

    :param url: str, the URL of the Wikipedia page to scrape
    :return: dict, a dictionary containing the title, text, and links of the Wikipedia page
    """
    soup = get_html(url)
    title = extract_title(soup)
    text = extract_text(soup)
    links = extract_links(soup)
    result = {'title': title, 'text': text, 'links': links}
    return result
